In [1]:
import fitz
import re 
import os
from docx import Document
from PIL import Image 
import matplotlib.pyplot as plt
from pytesseract import pytesseract
import pandas as pd 
import uuid
from datetime import datetime
from pptx import Presentation 
import numpy as np
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import sqlite3 as sql 
import json

### Text Extraction functions  

In [2]:
txt_content = []

In [3]:
# Text file reader 
def txt_file_reader(txt):
    
    
    with open(txt, 'r', encoding='utf-8') as f:
        print(f'the context the the file {txt}')
        lines = f.read()
        new_lines = lines.replace('\n','')
        txt_content.append(new_lines)
        return txt_content

In [4]:
# reading docx file 
def extract_docx_content(doc_path):
    try:
        if os.path.exists(doc_path):
            doc_obj = Document(doc_path)
            full_text = [paragraph.text for paragraph in doc_obj.paragraphs]
            return '\n'.join(full_text)
        else:
            print(f"No doc file found at: {doc_path}")
            return None
    except Exception as e:
        print(f'The error is: {e}')
        return None

In [5]:
# pdf reader
# txt_content = []
def get_pdf_content(pdf_file):
    
    pdf_obj = fitz.open(pdf_file)
    num_of_page = pdf_obj.page_count
    for i in range(0, num_of_page):
        page = pdf_obj.load_page(i)
        text_content = page.get_text()
        raw_text = text_content.replace('\n', '')  
        txt_content.append(raw_text)
        print(txt_content)
    return '\n'.join(txt_content)


In [6]:
# The thing should return the string insteda of list
# The coorrect the if and else statement's

In [7]:
# reading pptx 


def ppt_identifier(ppt):
    prs = Presentation(ppt)
    # txt_content = []
    for slide_num, slide in enumerate(prs.slides , start=1):
        slide_text = []
        print(f"\nSlide {slide_num}:")
        for shape in slide.shapes:
            if hasattr(shape,'text'):# gpt(The hasatt checks that the given attribute 'text' is present or not and if it is id present then extract the in that variabes the prnt it  )
                slide_text.append(shape.text.strip()) 
        
            full_string_text  = ''.join(slide_text)
            txt_content.append(full_string_text)
            return txt_content

In [8]:
# C:\Users\HP\AppData\Local\Programs\Tesseract-OCR'
def Ocr_identifier(img):
    txt_content=[]
    pytesseract.tesseract_cmd = r'C:\Users\HP\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
    image_path2=img
    image_o = Image.open(image_path2)
    text = pytesseract.image_to_string(image_o)
    # txt_conten.append(text)
    return text

### Text extracion fom the file 

In [9]:
folder_path = r"E:\AI Engineearing\Deep_learning\NLP_files\NLP_PROJECTS\API fetchiig wtih Flask and pyhton\New folder"
files = [f for f in os.listdir(folder_path) if os.path.exists(os.path.join(folder_path,f))]
print(files)

['Leading VS Lagging.pdf']


In [10]:

def file_unsopported_handeling(file_path):
    print(f"The file {os.path.basename(file_path) }is unsupported ⚠️ ")
    return None 
def text_extractor(f_path ,file_name):
    file_handlers = {
        
        '.txt': txt_file_reader,
        '.jpg': Ocr_identifier,
        '.png': Ocr_identifier,
        '.pdf': get_pdf_content,
        '.pptx': ppt_identifier,
        '.docx': extract_docx_content
    }
    extracted_text = []
    for file_tp in file_name:
        try:
        
            file_path = os.path.join(f_path,file_tp)
    
            extenion = os.path.splitext(file_tp.lower())[1]
            handelear = file_handlers.get(extenion,file_unsopported_handeling)
    
            ext_txt = handelear(file_path)
    
            extracted_text.append(ext_txt)
               
        except Exception as e:
            print(f"The error is {e}")
    return extracted_text
extracted_text = text_extractor(folder_path ,files)


['Leading VS Lagging    Notes on Practical Usage • Leading Metrics help drive decisions → trigger A/B tests, UI changes, or campaigns. • Lagging Metrics track outcomes → used to measure success and guide future investments. • Best practice: Use leading indicators to predict and course-correct before lagging outcomes are impacted.   How to Use This Table for Analysis 1. Choose metrics per stage — Evaluate 2–3 core metrics in each AARRR stage (e.g., CTR, Activation Rate, DAU/MAU, K-Factor, ARPU). 2. Label them as leading vs lagging — Leading guides proactive decisions, lagging reflects outcomes. 3. Set benchmarks and targets — Beat industry or internal historical KPIs. 4. Run experiments — A/B test changes to onboarding (Activation), share flows (Referral), pricing (Revenue), etc. 5. Create dashboards — Define alerts for KPI drift (e.g., Activation Rate < X%), track weekly/monthly. 6. Perform cohort analysis — Measure improvement by comparing before/after cohorts.    Checklist: What Make

### Text Chunking  

In [11]:
def chunking_of_para(text):
    chunk_size = 500
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

combined_text = extracted_text
# full_text = ''.join(combined_text)
parts = chunking_of_para(str(combined_text))
len(parts)
for i , chunk in enumerate(parts,start=1):
    print(f'chunk {i}\n{chunk}\n {len(chunk)}')

chunk 1
['Leading VS Lagging    Notes on Practical Usage • Leading Metrics help drive decisions → trigger A/B tests, UI changes, or campaigns. • Lagging Metrics track outcomes → used to measure success and guide future investments. • Best practice: Use leading indicators to predict and course-correct before lagging outcomes are impacted.   How to Use This Table for Analysis 1. Choose metrics per stage — Evaluate 2–3 core metrics in each AARRR stage (e.g., CTR, Activation Rate, DAU/MAU, K-Factor, ARPU). 
 500
chunk 2
2. Label them as leading vs lagging — Leading guides proactive decisions, lagging reflects outcomes. 3. Set benchmarks and targets — Beat industry or internal historical KPIs. 4. Run experiments — A/B test changes to onboarding (Activation), share flows (Referral), pricing (Revenue), etc. 5. Create dashboards — Define alerts for KPI drift (e.g., Activation Rate < X%), track weekly/monthly. 6. Perform cohort analysis — Measure improvement by comparing before/after cohorts.  

In [12]:
# # creating a data frame from the available data 
# # file uuiD ,  col1 file name , Data/Time , Len(text), Type(jpg, png etc) 
# # df2 = file id and chunk  
# import pandas as pd 

### File creatioon of the chunke data where 1st csv is having meta data and 2nd is having Text content  

In [13]:
import pandas as pd 
import uuid
from datetime import datetime
def file_meta_data(file_list, extracted_texts, chunks_list):
    
    csv_file = 'File_info.csv'
    whiter_header = not os.path.exists(csv_file)
    

    all_meta_data = []
    
    for file_name_and_ext in file_list:
        file_name = file_name_and_ext.split('.')[0]
        file_type = file_name_and_ext.split('.')[1]
        data = {
            'uuid':(uuid.uuid4()),        
            'File_Name': file_name,
            'File_Type': file_type,
            'Date_time': datetime.now().strftime('%x %X'),
            'len_of_text': len(str(extracted_texts)),          
            'Total_chunks': len(chunks_list)
        }

        all_meta_data.append(data)

        
    df = pd.DataFrame(all_meta_data)
       
    
    df.to_csv(csv_file, mode= 'a' , index = False,  header=whiter_header,columns=data.keys())
    
    return df
meta_data_file = file_meta_data(files ,text_extractor ,parts)
meta_data_file

,uuid,File_Name,File_Type,Date_time,len_of_text,Total_chunks
0,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2,Leading VS Lagging,pdf,08/02/25 00:44:53,47,25


In [14]:
# o File Content DataFrame:
# o Columns: chunkID (UUID), Datetime, chunk (text), fileID

In [15]:

def fiel_data_info (txt_chunks , file_uuid):
    
    content_file = 'File_content.csv'
    whiter_header = not os.path.exists(content_file)

    file_data = []
    for chunk in txt_chunks:
        
        content_storage = {
            'data_2_uuid':(uuid.uuid4()),
            'Date_and_time':datetime.now().strftime('%x %X'),
            'chunk': chunk.strip(),
            'fileID':file_uuid
        }
        file_data.append(content_storage)
    
    Chunk_data = pd.DataFrame(file_data)
    Chunk_data.to_csv(content_file,index=False, mode='a' ,header = whiter_header, columns=Chunk_data.columns)
    return Chunk_data
for i in meta_data_file.index:
    file_uuid =  meta_data_file.iloc[i]['uuid']
    chunk_info = fiel_data_info(parts , file_uuid)
chunk_info

,data_2_uuid,Date_and_time,chunk,fileID
0,4ad23332-6524-48b9-ace7-deb85858c62a,08/02/25 00:44:53,['Leading VS Lagging Notes on Practical Usa...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
1,c18bd703-fb10-4b02-96fe-8da20c4cf842,08/02/25 00:44:53,2. Label them as leading vs lagging — Leading ...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
2,d9b5d27d-aac4-45e4-8102-31c60b98147c,08/02/25 00:44:53,"s a Metric ""Good"" Criteria Why It Matters Ques...",d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
3,a8869a7a-58cc-43b0-bbfe-6391da77b855,08/02/25 00:44:53,"ful across versions, cohorts, or time periods ...",d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
4,d2fd5cb5-02b8-42ba-bd40-781e0687df3c,08/02/25 00:44:53,ese look impressive but don’t guide decisions:...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
5,2714bd3c-8d7b-4e6c-ac65-7366e4b42638,08/02/25 00:44:53,ss digital products at MAANG companies (Meta/A...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
6,345c23ff-5548-4046-ba88-152dbe4391dd,08/02/25 00:44:53,"iveness A/B test creatives, optimize targeting...",d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
7,3f27cc20-d602-4bf3-a8e6-614efaeea5ab,08/02/25 00:44:53,erm retention Trigger onboarding nudges if low...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
8,6aa85a9f-357d-4bf5-ab5c-90a583972a9b,08/02/25 00:44:53,Lagging % of users lost over time; survival of...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
9,4d8a630a-c061-43bb-9d3b-884c70acb059,08/02/25 00:44:53,vite Acceptance Rate Leading % of sent invites...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2


### Adding processing the text 
* usig the nltk
*  TFIDF vectorization
*  And storing the data in sql lite  

In [16]:
data = pd.read_csv(r'File_content.csv')
data

,data_2_uuid,Date_and_time,chunk,fileID
0,4ad23332-6524-48b9-ace7-deb85858c62a,08/02/25 00:44:53,['Leading VS Lagging Notes on Practical Usa...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
1,c18bd703-fb10-4b02-96fe-8da20c4cf842,08/02/25 00:44:53,2. Label them as leading vs lagging — Leading ...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
2,d9b5d27d-aac4-45e4-8102-31c60b98147c,08/02/25 00:44:53,"s a Metric ""Good"" Criteria Why It Matters Ques...",d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
3,a8869a7a-58cc-43b0-bbfe-6391da77b855,08/02/25 00:44:53,"ful across versions, cohorts, or time periods ...",d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
4,d2fd5cb5-02b8-42ba-bd40-781e0687df3c,08/02/25 00:44:53,ese look impressive but don’t guide decisions:...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
5,2714bd3c-8d7b-4e6c-ac65-7366e4b42638,08/02/25 00:44:53,ss digital products at MAANG companies (Meta/A...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
6,345c23ff-5548-4046-ba88-152dbe4391dd,08/02/25 00:44:53,"iveness A/B test creatives, optimize targeting...",d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
7,3f27cc20-d602-4bf3-a8e6-614efaeea5ab,08/02/25 00:44:53,erm retention Trigger onboarding nudges if low...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
8,6aa85a9f-357d-4bf5-ab5c-90a583972a9b,08/02/25 00:44:53,Lagging % of users lost over time; survival of...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2
9,4d8a630a-c061-43bb-9d3b-884c70acb059,08/02/25 00:44:53,vite Acceptance Rate Leading % of sent invites...,d3cce6b1-d720-4a3b-8ce6-1453ef00eeb2


## Performing NLTK taks
* TOkenize the chunk using the word tokinizer
* Removing the stopwords

In [17]:
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [18]:
def remove_punctuation(text):
    return "".join(char for char in text if char not in string.punctuation)
def task_1 (text):
    try:
        # converting to lowercase
        text = text.lower()
        text = remove_punctuation(text)
        # ccreating word tokens 
        stop_words = set(stopwords.words('english'))
        word_token = sent_tokenize(text,language='english') 
        clean_word =  [words for words in word_token if words  not in stop_words]

        return "".join(clean_word)
    except Exception as e:
        return f"The error is {e}"
cleaned_data = [task_1(i) for i in data['chunk']]
print((cleaned_data))


['leading vs lagging    notes on practical usage • leading metrics help drive decisions → trigger ab tests ui changes or campaigns • lagging metrics track outcomes → used to measure success and guide future investments • best practice use leading indicators to predict and coursecorrect before lagging outcomes are impacted   how to use this table for analysis 1 choose metrics per stage — evaluate 2–3 core metrics in each aarrr stage eg ctr activation rate daumau kfactor arpu', '2 label them as leading vs lagging — leading guides proactive decisions lagging reflects outcomes 3 set benchmarks and targets — beat industry or internal historical kpis 4 run experiments — ab test changes to onboarding activation share flows referral pricing revenue etc 5 create dashboards — define alerts for kpi drift eg activation rate  x track weeklymonthly 6 perform cohort analysis — measure improvement by comparing beforeafter cohorts    checklist what make', 's a metric good criteria why it matters questi

## Implementing word Tokinizer and sentence tokinizer ,lowering the sentence and removing punctuation

In [19]:
def remove_punctuation(text):
    return "".join(char for char in text if char not in string.punctuation)
def task_1 (text):
    try:
        # converting to lowercase
        text = text.lower()
        text = remove_punctuation(text)
        # ccreating word tokens 
        stop_words = set(stopwords.words('english'))
        word_token = sent_tokenize(text,language='english') 
        clean_word =  [words for words in word_token if words  not in stop_words]

        return "".join(clean_word)
    except Exception as e:
        return f"The error is {e}"
cleaned_data = [task_1(i) for i in data['chunk']]
print(cleaned_data)


['leading vs lagging    notes on practical usage • leading metrics help drive decisions → trigger ab tests ui changes or campaigns • lagging metrics track outcomes → used to measure success and guide future investments • best practice use leading indicators to predict and coursecorrect before lagging outcomes are impacted   how to use this table for analysis 1 choose metrics per stage — evaluate 2–3 core metrics in each aarrr stage eg ctr activation rate daumau kfactor arpu', '2 label them as leading vs lagging — leading guides proactive decisions lagging reflects outcomes 3 set benchmarks and targets — beat industry or internal historical kpis 4 run experiments — ab test changes to onboarding activation share flows referral pricing revenue etc 5 create dashboards — define alerts for kpi drift eg activation rate  x track weeklymonthly 6 perform cohort analysis — measure improvement by comparing beforeafter cohorts    checklist what make', 's a metric good criteria why it matters questi

### Implementing Tf-Idf Vactorization 

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [21]:
tf_idf_vactor = TfidfVectorizer()
tf_idf_vactor.fit(cleaned_data)
tfidf_data = tf_idf_vactor.transform(cleaned_data).toarray()

In [22]:
tfidf_data

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.10084404,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [23]:
tf_idf_vactor.get_feature_names_out()

array(['1730', '1retention', '30', 'aarrr', 'ab', 'acceptance',
       'accordingly', 'accounts', 'achieves', 'acquisition', 'acro',
       'across', 'action', 'actionable', 'actions', 'activation',
       'active', 'activity', 'actual', 'ad', 'add', 'ads', 'after',
       'against', 'alerts', 'algorithm', 'algorithmic', 'aligned',
       'alignment', 'all', 'allocate', 'analysis', 'analyze', 'anchor',
       'and', 'app', 'appeal', 'are', 'arppu', 'arpu', 'as', 'ask', 'at',
       'ational', 'attributed', 'attribution', 'audience', 'automated',
       'average', 'avg', 'avoid', 'awareness', 'back', 'balance', 'base',
       'based', 'beat', 'becoming', 'before', 'beforeafter', 'behavior',
       'benchmark', 'benchmarkable', 'benchmarks', 'best', 'better',
       'between', 'bio', 'book', 'boost', 'bounce', 'brand',
       'brandcreator', 'brands', 'breaks', 'budget', 'budgets', 'build',
       'bundling', 'business', 'but', 'by', 'cacltv', 'calculate',
       'calculated', 'campaign'

In [24]:
# df = pd.DataFrame(tfidf_data,columns=tf_idf_vactor.get_feature_names_out())
# df['full_sentence'] = cleaned_data
# df.shape

### store in sqlLite 

In [25]:
def datbase_creation_and_storing (nltk_data , tfidf_data):

    
    db_name = sql.connect('app_maindb.db')
  
    cus = db_name.cursor()
    
    cus.execute("DROP TABLE IF EXISTS CHUNKS_TFIDF")

    table_creation_query = """ 
        CREATE TABLE CHUNKS_TFIDF(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        chunk_txt TEXT NOT NULL , 
        tfidf_vector TEXT NOT NULL
        )
        """
    cus.execute(table_creation_query)
    records = []
    for cleaned_text ,vactors in zip(nltk_data , tfidf_data):
       try: 
            vector_json = json.dumps(vactors.tolist())

            records.append((cleaned_text ,vactors))
       except Exception as e :
           print (f"The error is {e}")
    cus.executemany("INSERT INTO CHUNKS_TFIDF (chunk_txt, tfidf_vector) VALUES (?,?)", records)
    print(f"Successfully inserted {len(cleaned_data)} records into app_maindb.db.")

    # cus.execute("SELECT * FROM CHUNKS_TFIDF")
    # for row in cus.fetchall():
    #     print(row,end='')

    db_name.commit()
    # db_name.close()


In [26]:
datbase_creation_and_storing(cleaned_data, tfidf_data )

Successfully inserted 25 records into app_maindb.db.


In [27]:
# db_name = sql.connect('app_maindb.db')
# cus = db_name.cursor()

In [28]:
# cus.execute("DROP TABLE IF EXISTS CHUNKS_TFIDF")

In [29]:
# table_creation_query = """ 
# CREATE TABLE CHUNKS_TFIDF(
# id INTEGER PRIMARY KEY AUTOINCREMENT,
# chunk_txt TEXT NOT NULL , 
# tfidf_vector REAL NOT NULL
# )
# """

In [30]:
# cus.execute(table_creation_query)


In [31]:
# for cleaned_text ,vactors in zip(cleaned_data,tfidf_data):
#     vector_json = json.dumps(vactors.tolist())

#     cus.execute("INSERT INTO CHUNKS_TFIDF (chunk_txt, tfidf_vector) VALUES (?,?)",(cleaned_text, vector_json))
#     # print(cleaned_text,vector_json)    


# print(f"Successfully inserted {len(cleaned_data)} records into '{db_name}'.")
# cus.execute("SELECT * FROM CHUNKS_TFIDF")
# for row in cus.fetchall():
#         print(row,end='')

# db_name.commit()
# # db_name.close()

In [32]:
# for text , vec in zip(tf_idf_vactor.get_feature_names_out(),tf_idf_vactor.idf_):
#     print(text, vec)

In [33]:
# print(tf_idf_vactor.idf_)